<a href="https://colab.research.google.com/github/ShinAsakawa/2015corona/blob/master/2024notebooks/2024_0425Logistic_regression_of_Olivetti_face_dataset_sifted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# オリベッティ顔データベースを用いた顔認識

* Date: 2024_0425
* Author: 浅川 伸一 educ0233@komazawa-u.ac.jp

* 自身の Google Drive にコピーした上で実行してください。

In [ ]:
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from sklearn.datasets import fetch_olivetti_faces
try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

data = fetch_olivetti_faces()
X, y = data.data, data.target

import IPython
isColab = 'google.colab' in str(IPython.get_ipython())

In [ ]:
import numpy as np
from sklearn.datasets import fetch_olivetti_faces

data = fetch_olivetti_faces()
X, y = data.data, data.target

print(f"目標とするクラス(画像中の人物の数) :{np.unique(y)}")


In [ ]:
nrows = 40   # nrows 人分のデータを表示
#nrows = 4
ncols = 10
fig, fig_axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols * 1.4, nrows * 1.4), constrained_layout=True)
# constrained_layout は subplot や 凡例やカラーバーなどの装飾を自動的に調整して，
# ユーザが要求する論理的なレイアウトをできるだけ維持しながら， 図ウィンドウに収まるようにします。

for i in range(nrows):
    for j in range(ncols):
        x = i * 10 + j
        fig_axes[i][j].imshow(X[x].reshape(64,64), cmap='gray')
        fig_axes[i][j].axis('off')
        fig_axes[i][j].set_title(f'num:{x}, ID:{y[x]}')

In [ ]:
num = 39
plt.imshow(X_sifted[num].reshape(64,64), cmap='gray')
plt.show()

plt.imshow(X[num].reshape(64,64), cmap='gray')
plt.show()

# 機械学習手法による顔認識

## データの分割，訓練データとテストデータ

データを 2 分割して，訓練データセットとテストデータセットに分割します。
分割した訓練データセットでモデルのパラメータを学習し，しかる後に，テストデータセットで，その汎化性能を評価します。
このとき，テストデータセットでの性能が高いモデルが良いモデルということになります。

オリベッティ顔データセットには， 各被験者の 10 枚の顔画像が含まれています。
このうち，例えば 90% を訓練データとし，10% をテストデータとして使用することを考えます。
各顔データの訓練画像とテスト画像の数が同じになるように stratify 機能を使用してます。
したがって，各被験者には 9 枚の訓練用画像と 1 枚のテスト用画像が用意されることになります。
訓練データとテストデータの割合は split_ratio 変更することができます。


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as sns  # ヒートマップ描画のために使用します

#from sklearn.decomposition import PCA
#from sklearn.svm import SVC
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# split_ratio = 0.1 としているので，訓練データ対テストデータが 9:1 になります
split_ratio = 0.1
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=split_ratio, stratify=y, random_state=42)
print(f'X_train 訓練画像のサイズ: {X_train.shape}')
print(f'y_train 教師信号データのサイズ: {y_train.shape}')

## ロジスティック回帰による予測

In [ ]:
params = {'max_iter':10 ** 3,
          'C':1e3,
          'penalty':'l2'}

model = LogisticRegression(**params)
model.fit(X_train, y_train)    # 訓練データを用いて線形判別分析モデルを訓練
y_hat = model.predict(X_test)  # テストデータを使って予測を行い結果を y_hat に格納
print(f"ロジスティック回帰を用いた分類精度: {metrics.accuracy_score(y_test, y_hat):.3f}")

# 混同行列の表示
plt.figure(figsize=(8,6))
sns.heatmap(metrics.confusion_matrix(y_test, y_hat))

# ロジスティック回帰の問題点

データを右にシフトすると，いままでのパラメータがおかしくなる。
このことを確かめてみましょう。

In [ ]:
sift = 3
X_sifted = X.copy()
for (x0, y0) in zip(X, X_sifted):
    _x = x0.reshape(64,64)
    _y = y0.reshape(64,64)
    for  (__x, __y) in zip(_x, _y):
        #print(__x.shape, __y.shape)
        __y[:-sift] = __x[sift:]
        __y[-sift:] = __x[:sift]

y_hat_sifted = model.predict(X_sifted)
#print(y_hat_sifted)

In [ ]:
print(y_hat)
print(y_test)

y_all = model.predict(X)
print(y_all)
print(y_hat_sifted)

hit = (np.array(y_all == y_hat_sifted) * 1).sum()
print(f'正解率: {hit/y_all.shape[0] * 100:.3f}%',
      f' 正答数:{hit}',
      f' 総データ数:{X.shape[0]}'
      )

In [ ]:
sns.heatmap(metrics.confusion_matrix(y, y_hat_sifted))

In [ ]:
plt.figure(figsize=(8,6))
#sns.heatmap(metrics.confusion_matrix(y_test, y_hat))
metrics.confusion_matrix(y, y_hat_sifted).shape